# Basic Datetimes

The Python standard library comes with a module `datetime` that handles working with dates and times.  It contains the classes `datetime`, `time`, `date`, and `timedelta` that allow constructing, representing and working with time-oriented objects.  However, issues about timezones are deferred until the next lesson of this course.

One oddity about the `datetime` module is shared by a few other (older) standard library modules like `glob`, `array`, `copy`, and `time` and `pprint`.  In each, the main names inside the module you wish to import has the same name as the module itself.  I often trip over whether I used `import glob` or `from glob import glob` at the top of my code.

In [1]:
# Import module itself; more verbose in later code
import datetime
from pprint import pprint

## Datetime

A `datetime` object represents a complete timestamp or "moment" in time, optionally including time zone information.  There are a number of different ways of constructing these objects and a number of formats they can be represented as.  The actual class constructor takes a tuple of different components of the instant in time.  For example we might want to represent the current time when the code is called.

In [2]:
ts0 = datetime.datetime.now()
ts0

datetime.datetime(2020, 8, 15, 16, 8, 11, 183949)

Or we can represent a specific time by providing numeric arguments for each component of the time.

In [3]:
ts1 = datetime.datetime(year=2021, month=1, day=20, 
                        hour=12, minute=0, second=0, 
                        microsecond=123_456)
ts1

datetime.datetime(2021, 1, 20, 12, 0, 0, 123456)

In [4]:
# Parameters may be passed purely positionally
ts2 = datetime.datetime(2021, 1, 20, 12, 0, 0, 345_678)
ts2

datetime.datetime(2021, 1, 20, 12, 0, 0, 345678)

Often it is useful to create datetimes from or export to popular string representations. ISO 8601 format is the best choice, of course, but your requirement might be different.

In [5]:
# Equivalent: '2021-01-20T12:00:00.678901'
ts3 = datetime.datetime.fromisoformat('2021-01-20 12:00:00.345678')
ts3

datetime.datetime(2021, 1, 20, 12, 0, 0, 345678)

In [6]:
ts0.isoformat()

'2020-08-15T16:08:11.183949'

Custom formats can be defined using the format codes from C-family languages (ANSI X3.159-1989).  For Python, you can read documentation at:

> https://docs.python.org/3/library/datetime.html#strftime-strptime-behavior

There are a large number of these format codes, many of them locale aware.  For example, `%A` will produce one of "Sunday", "Monday", etc. in the locale `en_US`, but will produce "Sonntag", "Montag", etc. for `de_DE`.  The names `strptime()` and `strftime()` are also taken from that old C standard.

An example of output.

In [7]:
import locale
print(locale.getdefaultlocale())

fmt = '%A %b %d; %Y; %I:%M:%S %p'
datetime.datetime.strftime(ts0, fmt)

('en_US', 'UTF-8')


'Saturday Aug 15; 2020; 04:08:11 PM'

Reading in these formats uses the same codes.  The module is only semi-smart about validation.

In [8]:
# Good datetime object
strptime = datetime.datetime.strptime
strptime('Wednesday Aug 12; 2020; 05:29:12 PM', fmt)

datetime.datetime(2020, 8, 12, 17, 29, 12)

In [9]:
# Still "good" even though wrong weekday
strptime('Monday Aug 12; 2020; 05:29:12 PM', fmt)

datetime.datetime(2020, 8, 12, 17, 29, 12)

Some validation is performed though.

In [10]:
# It DOES check that it is a day name at all
try:
    strptime('Humpday Aug 08; 2020; 05:29:12 PM', fmt)
except ValueError as err:
    pprint(str(err), width=54)

("time data 'Humpday Aug 08; 2020; 05:29:12 PM' does "
 "not match format '%A %b %d; %Y; %I:%M:%S %p'")


In [11]:
# Day number not in month
try:
    strptime('Thursday Sep 31; 2020; 05:29:12 PM', fmt)
except ValueError as err:
    print(err)

day is out of range for month


## Comparing Datetimes

Datetime objects are comparable and hashable.  This makes it easy both to compare different datetimes and to use these objects as elements in sets or keys in dictionaries.  When you try out this notebook (at a time later than I compose it) you might get a different answer, but it will be Boolean either way:

Inequalities and equality:

In [12]:
print(ts0, ts1, sep='\n')
ts0 < ts1

2020-08-15 16:08:11.183949
2021-01-20 12:00:00.123456


True

In [13]:
print(ts2, ts3, sep='\n')
ts2 == ts3

2021-01-20 12:00:00.345678
2021-01-20 12:00:00.345678


True


Datetime objects are hashable.  Equality, of course, means that only one equal object is in a set.

In [14]:
dates = [ts0, ts1, ts2, ts3]
objs = {id(ts) for ts in dates}
distinct = set(dates)
print(objs)
distinct

{140149750458912, 140149750513408, 140149750512784, 140149750512112}


{datetime.datetime(2020, 8, 15, 16, 8, 11, 183949),
 datetime.datetime(2021, 1, 20, 12, 0, 0, 123456),
 datetime.datetime(2021, 1, 20, 12, 0, 0, 345678)}

Similarly with dictionary keys.

In [15]:
{ts0: "Hope", ts1: "Change", ts2: "Progress"}

{datetime.datetime(2020, 8, 15, 16, 8, 11, 183949): 'Hope',
 datetime.datetime(2021, 1, 20, 12, 0, 0, 123456): 'Change',
 datetime.datetime(2021, 1, 20, 12, 0, 0, 345678): 'Progress'}

## Date and Time

The separate classes `date` and `time` represent those named components of an overall `datetime`.  However, these time-related datatypes are simply different, and require explicit conversion to or from a `datetime`.  As with full datetime objects, we can construct dates and times in a number of ways.

The most basic constructors just give componets.

In [16]:
print("Date:", datetime.date(1941, 12, 7))
print("Time:", datetime.time(7, 55, 0))

Date: 1941-12-07
Time: 07:55:00


One way of constructing a date (or a datetime) is in terms of "seconds since the beginning of (Unix) time."  For a date alone, this only gets you "some time during that day."

In [17]:
import time
now = time.time()
# Seconds since the epoch
print(f" Seconds: {now:,}")
# What timestamp is that?
print("Datetime:", datetime.datetime.fromtimestamp(now))
# What day is that?
print("    Date:", datetime.date.fromtimestamp(now))

 Seconds: 1,597,522,091.399423
Datetime: 2020-08-15 16:08:11.399423
    Date: 2020-08-15


If we pick an arbitrary number, lets say "0" as the seconds-since-epoch, we will get a date (or datetime; but **not** a time for this method).

In [18]:
datetime.date.fromtimestamp(0)

datetime.date(1969, 12, 31)

This is a somewhat odd date, given that Unix time started at the stroke of midnight on January 1, 1970.  The key thing is Unix time start in UTC timezone, and my local time writing this is EST.  Hold on for the next lesson, it is a wild ride.

Negative numbers are perfectly fine, by the way.

In [19]:
long_ago = datetime.date.fromtimestamp(-6_857_300_000)
long_ago

datetime.date(1752, 9, 12)

This gives us another fun date, in that in England, and all its colonies of the time, the calendar was adjusted in 1752 between the Julian and Gregorian calendars, meaning that the dates between September 2 and September 14 did not occur that year.  Most of the rest of Europe had made the switch back in 1582, but there were some quibbles, and large wars, going on in that part of the world, known as the "Reformation" and "Counter-Reformation."

Calendrics are a strange and confusing story. We might jump forward from that date that perhaps did not exist to my own birth date.

In [20]:
david_day = long_ago.replace(year=1964)
david_day

datetime.date(1964, 9, 12)

## Converting Among Time Objects

We need to use explicit methods to move among the different kinds of date/time objects.  A datetime is the more universal object; I rarely bother with a date or time alone except perhaps as a final conversion step.

In [21]:
print(" Now:", ts0)
print("Date:", ts0.date())
print("Time:", ts0.time())

 Now: 2020-08-15 16:08:11.183949
Date: 2020-08-15
Time: 16:08:11.183949


From a datetime it is perfectly sensible to extract a date or a time, but the other direction is either lossy or impossible.  In concept, you could destructure component to generate a datetime.

In [22]:
y, mon, d = long_ago.year, long_ago.month, long_ago.day
h, m, s = (12, 1, 30)
datetime.datetime(y, mon, d, h, m, s).isoformat()

'1752-09-12T12:01:30'

In practice, we have a method to put these together.

In [23]:
combine = datetime.datetime.combine
combine(long_ago, datetime.time(12, 1, 30))

datetime.datetime(1752, 9, 12, 12, 1, 30)

## Timedelta

One thing that is often important is measuring the distance between datetimes.  To work with durations—or "deltas"—we use the `timedelta` class.  This is also produced as the result of subtraction between objects.

In [24]:
thousand_days = datetime.timedelta(days=1000)
print(thousand_days)

1000 days, 0:00:00


In [25]:
five_minutes = datetime.timedelta(minutes=5)
print(five_minutes)

0:05:00


Timedeltas can be added of subtracted from each other, with reasonable behavior.

In [26]:
after_a_while = thousand_days + five_minutes
print(after_a_while)

1000 days, 0:05:00


In [27]:
a_bit_less = thousand_days - five_minutes
a_bit_less

datetime.timedelta(days=999, seconds=86100)

In [28]:
print(3 * after_a_while)

3000 days, 0:15:00


Equalities and inequalities behave as you would expect.

In [29]:
after_a_while == datetime.timedelta(days=1000, minutes=5)

True

In [30]:
a_bit_less < after_a_while

True

There are a number of units available in constructing timedeltas, but the most coarse is days, not months, years, or centuries.  Units are not limited to the number of their cyclicities (24 hours, 60 seconds, etc).

In [31]:
hours_50k = datetime.timedelta(hours=50_000)
print(hours_50k)

2083 days, 8:00:00


In [32]:
hours_50k <= thousand_days + thousand_days

False

### Addition and Subtraction

Subtracting one datetime, or date, from another produces a timedelta.  Long increments will account for leap days where relevant, even though the units will not include years (just more days).  For days, the delta will be whole days only, and other units will not appear.

In [33]:
david_day - long_ago

datetime.timedelta(days=77431)

In [34]:
ts0 - ts1

datetime.timedelta(days=-158, seconds=14891, microseconds=60493)

Using times is more limited, correctly so.  Subtracting times seems like it makes sense, but of course they roll over at midnight each night, which causes problems.

In [35]:
t1 = datetime.time(hour=18, minute=25, second=35)
t2 = datetime.time(hour=20, minute=0, second=0)
try:
    t2 - t1
except TypeError as err:
    print(err)

unsupported operand type(s) for -: 'datetime.time' and 'datetime.time'


The solution is to use datetimes rather than only times.

In [36]:
today = datetime.date.today()
ts4 = combine(today, t1)
ts5 = combine(today, t2)
print("Waiting:", ts5 - ts4)

Waiting: 1:34:25


Code like that just above could go wrong if times you want the difference between cross midnight.  This is something that can commonly occur, for example, when writing to log files. However, the solution is simple.  Do not fix "today" as I did above, but rather call `today()` at the moment the entry is created.  

In other cases, it will not be "today" you want; but just think about appropriate date components.

In [37]:
tomorrow = today + datetime.timedelta(days=1)
late_night = datetime.time(23, 59, 30)
early_next = datetime.time(0, 2, 15)
print(late_night)
print(early_next)

23:59:30
00:02:15


In [38]:
ts6 = combine(today, late_night)
ts7 = combine(tomorrow, early_next)
ts7 - ts6

datetime.timedelta(seconds=165)

Often the use of a timedelta is not simply describing a duration, but also *applying* it to an existing datetime or date.  We cannot similarly apply it to a time because of the wrap-around issues (even though some such operations would not wrap, it is excluded at the level of the datatype).

In [39]:
print(ts1)
print(after_a_while)
ts1 + after_a_while

2021-01-20 12:00:00.123456
1000 days, 0:05:00


datetime.datetime(2023, 10, 17, 12, 5, 0, 123456)

Likewise for a date.

In [40]:
print(today)
print(today + after_a_while)

2020-08-15
2023-05-12


Some deltas will not change the date.

In [41]:
print(today + datetime.timedelta(seconds=1))

2020-08-15


But not for a time. Not even one that is not near midnight.

In [42]:
print(t1)
try: 
    t1 + datetime.timedelta(seconds=1)
except Exception as err:
    print(err)

18:25:35
unsupported operand type(s) for +: 'datetime.time' and 'datetime.timedelta'
